# Module 7: Optimization and Gradient Descent

**Goal:** Understand how gradient descent works, experiment with learning rates and optimizers.

**Prerequisites:** Module 3 (Linear Regression), Module 4 (Logistic Regression)

**Expected Runtime:** ~20 minutes

**Outputs:**
- Visualizations of gradient descent paths
- Comparison of optimizers on different loss surfaces
- Understanding of learning rate effects

---

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

# Plot style
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## Part 1: Understanding the Loss Surface

Gradient descent tries to find the lowest point on a "loss surface" - a landscape where the height represents how bad the model's predictions are.

In [ ]:
# Define a simple quadratic loss surface (bowl shape)
def quadratic_loss(x, y):
    return x**2 + y**2

def quadratic_gradient(x, y):
    return np.array([2*x, 2*y])

# Visualize the loss surface
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
Z = quadratic_loss(X, Y)

fig = plt.figure(figsize=(14, 5))

# 3D view
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8)
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')
ax1.set_zlabel('Loss')
ax1.set_title('3D View: Quadratic Bowl')

# Contour view
ax2 = fig.add_subplot(122)
contour = ax2.contour(X, Y, Z, levels=15, cmap='viridis')
ax2.scatter([0], [0], color='green', s=100, marker='*', label='Minimum')
ax2.set_xlabel('θ₁')
ax2.set_ylabel('θ₂')
ax2.set_title('Contour View (Bird\'s Eye)')
ax2.legend()
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print("The goal of gradient descent: Start somewhere on this surface and find the minimum (green star).")

## Part 2: Implementing Gradient Descent from Scratch

Let's implement vanilla gradient descent to see exactly what happens at each step.

In [ ]:
def gradient_descent(start, gradient_fn, loss_fn, lr=0.1, n_steps=50):
    """
    Vanilla gradient descent.
    
    Args:
        start: Starting position [x, y]
        gradient_fn: Function that returns gradient at a point
        loss_fn: Function that returns loss at a point
        lr: Learning rate
        n_steps: Number of steps to take
    
    Returns:
        path: List of positions visited
        losses: List of loss values
    """
    position = np.array(start, dtype=float)
    path = [position.copy()]
    losses = [loss_fn(position[0], position[1])]
    
    for _ in range(n_steps):
        grad = gradient_fn(position[0], position[1])
        position = position - lr * grad  # The core update rule!
        path.append(position.copy())
        losses.append(loss_fn(position[0], position[1]))
    
    return np.array(path), np.array(losses)

In [ ]:
# Run gradient descent with a reasonable learning rate
start = [-2.5, 2.5]
path, losses = gradient_descent(start, quadratic_gradient, quadratic_loss, lr=0.1, n_steps=30)

# Visualize the path
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Path on contour plot
ax1 = axes[0]
ax1.contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.7)
ax1.plot(path[:, 0], path[:, 1], 'o-', color='red', markersize=4, linewidth=1.5, label='GD Path')
ax1.scatter([start[0]], [start[1]], color='red', s=100, marker='s', zorder=5, label='Start')
ax1.scatter([0], [0], color='green', s=100, marker='*', zorder=5, label='Minimum')
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')
ax1.set_title(f'Gradient Descent Path (lr={0.1})')
ax1.legend()
ax1.set_aspect('equal')

# Loss over time
ax2 = axes[1]
ax2.plot(losses, 'b-', linewidth=2)
ax2.set_xlabel('Step')
ax2.set_ylabel('Loss')
ax2.set_title('Loss Over Time')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Starting loss: {losses[0]:.4f}")
print(f"Final loss: {losses[-1]:.4f}")
print(f"Final position: ({path[-1, 0]:.4f}, {path[-1, 1]:.4f})")

## Part 3: The Learning Rate Effect

Learning rate is the most important hyperparameter in gradient descent. Let's see what happens with different values.

In [ ]:
# TODO: Experiment with different learning rates
# Try: 0.01, 0.1, 0.5, 0.95, 1.1
# Predict what will happen before running!

learning_rates = [0.01, 0.1, 0.5, 0.95, 1.1]
colors = ['blue', 'green', 'orange', 'red', 'purple']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Path visualization
ax1 = axes[0]
ax1.contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.5)

# Loss curves
ax2 = axes[1]

for lr, color in zip(learning_rates, colors):
    path, losses = gradient_descent(start, quadratic_gradient, quadratic_loss, lr=lr, n_steps=30)
    
    # Clip for visualization (some might explode)
    path = np.clip(path, -5, 5)
    losses = np.clip(losses, 0, 50)
    
    ax1.plot(path[:, 0], path[:, 1], 'o-', color=color, markersize=3, 
             linewidth=1, label=f'lr={lr}', alpha=0.7)
    ax2.plot(losses, color=color, linewidth=2, label=f'lr={lr}')

ax1.scatter([0], [0], color='green', s=100, marker='*', zorder=5)
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')
ax1.set_title('Paths with Different Learning Rates')
ax1.legend()
ax1.set_xlim(-4, 4)
ax1.set_ylim(-4, 4)

ax2.set_xlabel('Step')
ax2.set_ylabel('Loss')
ax2.set_title('Loss Curves')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 20)

plt.tight_layout()
plt.show()

### Observations

**lr = 0.01:** Too slow - takes many steps to make progress  
**lr = 0.1:** Just right - smooth convergence  
**lr = 0.5:** Getting aggressive - some oscillation  
**lr = 0.95:** Barely stable - oscillating around minimum  
**lr = 1.1:** Diverging! - overshooting and exploding  

## Part 4: The Ravine Problem

Real loss surfaces aren't nice bowls. Let's see what happens with a narrow "ravine" shape.

In [ ]:
# Ravine: Much steeper in y-direction than x-direction
def ravine_loss(x, y):
    return 0.5 * x**2 + 10 * y**2

def ravine_gradient(x, y):
    return np.array([x, 20*y])

# Visualize the ravine
Z_ravine = ravine_loss(X, Y)

plt.figure(figsize=(8, 6))
plt.contour(X, Y, Z_ravine, levels=20, cmap='viridis')
plt.colorbar(label='Loss')
plt.xlabel('θ₁')
plt.ylabel('θ₂')
plt.title('Ravine Loss Surface: Steep in Y, Flat in X')
plt.scatter([0], [0], color='green', s=100, marker='*', label='Minimum')
plt.legend()
plt.show()

In [ ]:
# Run vanilla SGD on the ravine
start_ravine = [-2.5, 0.5]
path_sgd, losses_sgd = gradient_descent(start_ravine, ravine_gradient, ravine_loss, lr=0.05, n_steps=50)

plt.figure(figsize=(10, 6))
plt.contour(X, Y, Z_ravine, levels=20, cmap='viridis', alpha=0.5)
plt.plot(path_sgd[:, 0], path_sgd[:, 1], 'o-', color='red', markersize=4, linewidth=1, label='SGD Path')
plt.scatter([start_ravine[0]], [start_ravine[1]], color='red', s=100, marker='s', zorder=5)
plt.scatter([0], [0], color='green', s=100, marker='*', zorder=5)
plt.xlabel('θ₁')
plt.ylabel('θ₂')
plt.title('SGD on Ravine: Zigzag Problem!')
plt.legend()
plt.show()

print("Notice the zigzag pattern! SGD oscillates across the narrow valley.")

## Part 5: Momentum to the Rescue

Momentum helps smooth out the oscillations by accumulating velocity from past gradients.

In [ ]:
def gradient_descent_momentum(start, gradient_fn, loss_fn, lr=0.05, momentum=0.9, n_steps=50):
    """
    SGD with momentum.
    """
    position = np.array(start, dtype=float)
    velocity = np.zeros(2)
    path = [position.copy()]
    losses = [loss_fn(position[0], position[1])]
    
    for _ in range(n_steps):
        grad = gradient_fn(position[0], position[1])
        velocity = momentum * velocity - lr * grad  # Accumulate velocity
        position = position + velocity
        path.append(position.copy())
        losses.append(loss_fn(position[0], position[1]))
    
    return np.array(path), np.array(losses)

In [ ]:
# Compare SGD vs Momentum on the ravine
path_momentum, losses_momentum = gradient_descent_momentum(start_ravine, ravine_gradient, ravine_loss, 
                                                           lr=0.05, momentum=0.9, n_steps=50)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Paths
ax1 = axes[0]
ax1.contour(X, Y, Z_ravine, levels=20, cmap='viridis', alpha=0.5)
ax1.plot(path_sgd[:, 0], path_sgd[:, 1], 'o-', color='red', markersize=3, 
         linewidth=1, label='SGD', alpha=0.7)
ax1.plot(path_momentum[:, 0], path_momentum[:, 1], 'o-', color='blue', markersize=3, 
         linewidth=1, label='SGD + Momentum')
ax1.scatter([0], [0], color='green', s=100, marker='*', zorder=5)
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')
ax1.set_title('SGD vs Momentum on Ravine')
ax1.legend()

# Loss curves
ax2 = axes[1]
ax2.plot(losses_sgd, 'r-', linewidth=2, label='SGD')
ax2.plot(losses_momentum, 'b-', linewidth=2, label='SGD + Momentum')
ax2.set_xlabel('Step')
ax2.set_ylabel('Loss')
ax2.set_title('Loss Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

print(f"SGD final loss: {losses_sgd[-1]:.6f}")
print(f"Momentum final loss: {losses_momentum[-1]:.6f}")
print(f"\nMomentum converges ~{losses_sgd[-1]/losses_momentum[-1]:.1f}x closer to the minimum!")

## Part 6: Implementing Adam

Adam combines momentum with adaptive learning rates per parameter.

In [ ]:
def adam(start, gradient_fn, loss_fn, lr=0.1, beta1=0.9, beta2=0.999, epsilon=1e-8, n_steps=50):
    """
    Adam optimizer.
    """
    position = np.array(start, dtype=float)
    m = np.zeros(2)  # First moment (momentum)
    v = np.zeros(2)  # Second moment (adaptive)
    path = [position.copy()]
    losses = [loss_fn(position[0], position[1])]
    
    for t in range(1, n_steps + 1):
        grad = gradient_fn(position[0], position[1])
        
        # Update biased first moment estimate
        m = beta1 * m + (1 - beta1) * grad
        # Update biased second moment estimate
        v = beta2 * v + (1 - beta2) * grad**2
        
        # Bias correction
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)
        
        # Update position
        position = position - lr * m_hat / (np.sqrt(v_hat) + epsilon)
        
        path.append(position.copy())
        losses.append(loss_fn(position[0], position[1]))
    
    return np.array(path), np.array(losses)

In [ ]:
# Compare all three optimizers on the ravine
path_adam, losses_adam = adam(start_ravine, ravine_gradient, ravine_loss, lr=0.3, n_steps=50)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Paths
ax1 = axes[0]
ax1.contour(X, Y, Z_ravine, levels=20, cmap='viridis', alpha=0.5)
ax1.plot(path_sgd[:, 0], path_sgd[:, 1], 'o-', color='red', markersize=3, 
         linewidth=1, label='SGD', alpha=0.6)
ax1.plot(path_momentum[:, 0], path_momentum[:, 1], 'o-', color='blue', markersize=3, 
         linewidth=1, label='Momentum', alpha=0.6)
ax1.plot(path_adam[:, 0], path_adam[:, 1], 'o-', color='purple', markersize=3, 
         linewidth=1.5, label='Adam')
ax1.scatter([0], [0], color='green', s=100, marker='*', zorder=5)
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')
ax1.set_title('SGD vs Momentum vs Adam')
ax1.legend()

# Loss curves
ax2 = axes[1]
ax2.plot(losses_sgd, 'r-', linewidth=2, label='SGD')
ax2.plot(losses_momentum, 'b-', linewidth=2, label='Momentum')
ax2.plot(losses_adam, 'purple', linewidth=2, label='Adam')
ax2.set_xlabel('Step')
ax2.set_ylabel('Loss')
ax2.set_title('Loss Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

print("Final losses:")
print(f"  SGD:      {losses_sgd[-1]:.6f}")
print(f"  Momentum: {losses_momentum[-1]:.6f}")
print(f"  Adam:     {losses_adam[-1]:.6f}")

## Part 7: Regularization Effect

Regularization adds a penalty for large weights, effectively changing the loss surface.

In [ ]:
# Original loss vs L2 regularized loss
lambda_l2 = 0.5

def l2_regularized_loss(x, y):
    return quadratic_loss(x, y) + lambda_l2 * (x**2 + y**2)

# Visualize the effect
Z_l2 = l2_regularized_loss(X, Y)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax1 = axes[0]
c1 = ax1.contour(X, Y, Z, levels=15, cmap='viridis')
ax1.set_title('Original Loss Surface')
ax1.set_xlabel('θ₁')
ax1.set_ylabel('θ₂')

ax2 = axes[1]
c2 = ax2.contour(X, Y, Z_l2, levels=15, cmap='viridis')
ax2.set_title(f'With L2 Regularization (λ={lambda_l2})')
ax2.set_xlabel('θ₁')
ax2.set_ylabel('θ₂')

plt.tight_layout()
plt.show()

print("L2 regularization makes the bowl steeper, encouraging smaller weights.")
print("This helps prevent overfitting by penalizing extreme parameter values.")

## Part 8: TODO - Find the Optimal Learning Rate

A common technique is to try several learning rates and pick the one that converges fastest without diverging.

In [ ]:
# TODO: Complete this experiment
# Test learning rates: [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
# For each, record: final loss, number of steps to reach loss < 0.01, whether it diverged

test_lrs = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
results = []

for lr in test_lrs:
    path, losses = gradient_descent(start, quadratic_gradient, quadratic_loss, lr=lr, n_steps=100)
    
    # Check for divergence
    diverged = np.any(np.isnan(losses)) or np.any(losses > 1000)
    
    # Steps to reach low loss
    steps_to_converge = None
    for i, loss in enumerate(losses):
        if loss < 0.01:
            steps_to_converge = i
            break
    
    results.append({
        'lr': lr,
        'final_loss': losses[-1] if not diverged else float('inf'),
        'steps_to_0.01': steps_to_converge,
        'diverged': diverged
    })

# TODO: Print results and identify the best learning rate
print("Learning Rate Experiment Results:")
print("-" * 50)
for r in results:
    status = "DIVERGED" if r['diverged'] else f"loss={r['final_loss']:.6f}"
    steps = r['steps_to_0.01'] if r['steps_to_0.01'] else "Never"
    print(f"lr={r['lr']:<6} | {status:<20} | Steps to 0.01: {steps}")

# TODO: Which learning rate would you choose and why?
# Write your answer below:

# YOUR ANSWER:
# Best LR: ___
# Reason: ___

## Part 9: Connecting to StreamCart

Let's apply what we learned about optimizers to our churn prediction model. When training a neural network or logistic regression on StreamCart data, these concepts matter:

In [ ]:
# Applying optimizer concepts to real ML training
# This code demonstrates how optimizer choices affect model training

import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load StreamCart data
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'
try:
    customers = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
except:
    print("Using simulated data for demonstration")
    np.random.seed(42)
    customers = pd.DataFrame({
        'tenure_days': np.random.randint(1, 365, 1000),
        'orders_total': np.random.randint(0, 50, 1000),
        'total_spend': np.random.uniform(0, 5000, 1000),
        'churn_30d': np.random.binomial(1, 0.2, 1000)
    })

# Prepare features
feature_cols = ['tenure_days', 'orders_total', 'total_spend']
available = [c for c in feature_cols if c in customers.columns]
X = customers[available].fillna(0)
y = customers['churn_30d']

# Scale features (critical for gradient descent!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Compare different learning rates with SGD
print("=== Learning Rate Impact on StreamCart Churn Model ===\n")
for lr in [0.0001, 0.001, 0.01, 0.1, 1.0]:
    # SGDClassifier uses gradient descent under the hood
    model = SGDClassifier(
        loss='log_loss',       # Logistic regression
        learning_rate='constant',
        eta0=lr,               # Learning rate
        max_iter=100,
        random_state=42
    )
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"lr={lr:<6} → Test Accuracy: {acc:.1%}")

print("\n💡 Insight: Too high or too low learning rate hurts performance!")
print("   The best lr depends on feature scaling and data characteristics.")

## Part 10: Stakeholder Summary

**TODO:** Write a 3-bullet summary (~100 words) explaining optimization to a non-technical colleague:
• **What gradient descent does:** [Plain English explanation]
• **Why learning rate matters:** [One sentence on what happens if too high/low]
• **When to use Adam vs SGD:** [Simple guidance]

### Your Summary:

*Write your summary here...*

---

## Self-Check

Uncomment and run the asserts below to verify your optimizer experiments are correct.

In [ ]:
# SELF-CHECK: Verify your optimization experiments
# Run this cell after completing the TODOs above

assert len(losses) > 1 and losses[-1] < losses[0], "Loss should decrease over gradient descent steps"
assert len(results) > 0, "Learning rate experiment results should exist"
converged = [r for r in results if not r.get('diverged', False)]
assert len(converged) >= 2, "At least 2 learning rates should converge"
best = min(converged, key=lambda r: r['final_loss'])
assert best['final_loss'] < 0.5, f"Best learning rate should reach low loss, got {best['final_loss']:.3f}"
print(f"✅ Self-check passed! {len(converged)}/{len(results)} learning rates converged")

## Key Takeaways

1. **Gradient descent** is how models learn - iteratively adjusting parameters to reduce error
2. **Learning rate** is critical: too high → diverge, too low → slow
3. **Momentum** helps smooth oscillations on narrow valleys
4. **Adam** is the default choice: combines momentum + adaptive learning rates
5. **Regularization** changes the loss surface to prefer smaller weights

### Next Steps
- Explore the interactive playground to visualize different surfaces and optimizers
- Complete the quiz to test your understanding